In [ ]:
!pip install numpy opencv-python


In [1]:
import numpy as np
import cv2 as cv
import glob

## 1. Calibration images:

### a)

In [5]:
# Termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# If 12x9 squares, use 11x8 inner corners
PATTERN_SIZE = (11, 8)  # Inner corners, not squares
SQUARE_SIZE = 15  # mm

# Prepare object points
objp = np.zeros((PATTERN_SIZE[0] * PATTERN_SIZE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:PATTERN_SIZE[0], 0:PATTERN_SIZE[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE  # Scale by actual square size

# Dictionary to store all calibration results
calibration_results = {}

# Process each calibration folder
calib_folders = ['assign1_data/Calibration/calib1/*.png',
                 'assign1_data/Calibration/calib2/*.png',
                 'assign1_data/Calibration/calib3/*.png']

for folder_pattern in calib_folders:
    objpoints_temp = []
    imgpoints_temp = []
    
    images = glob.glob(folder_pattern)
    
    for fname in images:
        img = cv.imread(fname)
            
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # Find chessboard corners
        ret, corners = cv.findChessboardCorners(
            gray, 
            PATTERN_SIZE,
            cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE
        )
        
        if ret:
            objpoints_temp.append(objp)
            # Refine corner positions to subpixel accuracy
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints_temp.append(corners2)
    
    # Calibrate if corners were found
    if len(objpoints_temp) > 0:
        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
            objpoints_temp, 
            imgpoints_temp, 
            gray.shape[::-1], 
            None, 
            None
        )
        
        # Calculate reprojection error
        mean_error = 0
        for i in range(len(objpoints_temp)):
            imgpoints2, _ = cv.projectPoints(objpoints_temp[i], rvecs[i], tvecs[i], mtx, dist)
            error = cv.norm(imgpoints_temp[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
            mean_error += error
        
        reprojection_error = mean_error / len(objpoints_temp)
        
        # Store results
        calib_name = folder_pattern.split('/')[-2]  # Extract 'calib1', 'calib2', etc.
        calibration_results[calib_name] = {
            'mtx': mtx,
            'dist': dist,
            'rvecs': rvecs,
            'tvecs': tvecs,
            'reprojection_error': reprojection_error,
            'num_images': len(objpoints_temp)
        }
        
        print(f"\n{'='*50}")
        print(f"Calibration Results for {folder_pattern}")
        print(f"{'='*50}")
        print(f"\nIntrinsic Matrix (K):")
        print(mtx)
        print(f"\nDistortion Coefficients (k1, k2, p1, p2, k3):")
        print(dist)
        print(f"\nReprojection Error: {reprojection_error:.4f} pixels")
        print(f"{'='*50}\n")
    else:
        print(f"⚠ WARNING: No corners found in any images from {folder_pattern}\n")

cv.destroyAllWindows()

# ============================================================================
# SELECT BEST CALIBRATION BASED ON LOWEST REPROJECTION ERROR
# ============================================================================

if len(calibration_results) == 0:
    print("ERROR: No successful calibrations found!")
    exit()

# Find calibration with lowest reprojection error
best_calib = min(calibration_results.items(), 
                 key=lambda x: x[1]['reprojection_error'])
best_calib_name = best_calib[0]
best_calib_data = best_calib[1]

print("\n" + "="*60)
print("BEST CALIBRATION SELECTION")
print("="*60)
print(f"\nCalibration comparison:")
for name, data in sorted(calibration_results.items()):
    marker = " ← SELECTED (Best)" if name == best_calib_name else ""
    print(f"  {name}: Reprojection Error = {data['reprojection_error']:.4f} pixels "
          f"({data['num_images']} images){marker}")

print(f"\nUsing {best_calib_name} for extrinsic calibration")
print(f"Reprojection Error: {best_calib_data['reprojection_error']:.4f} pixels")
print("="*60 + "\n")

# Load the best intrinsic calibration results
mtx = best_calib_data['mtx']
dist = best_calib_data['dist']

print("Intrinsic Matrix (K):")
print(mtx)
print("\nDistortion Coefficients:")
print(dist)
print()


Calibration Results for assign1_data/Calibration/calib1/*.png

Intrinsic Matrix (K):
[[827.57845883   0.         334.83392431]
 [  0.         827.43748678 281.31041341]
 [  0.           0.           1.        ]]

Distortion Coefficients (k1, k2, p1, p2, k3):
[[-0.43653613  0.40330687 -0.00135039  0.00154944 -0.5335798 ]]

Reprojection Error: 0.0404 pixels


Calibration Results for assign1_data/Calibration/calib2/*.png

Intrinsic Matrix (K):
[[839.07613989   0.         339.34164645]
 [  0.         839.32227807 276.31090479]
 [  0.           0.           1.        ]]

Distortion Coefficients (k1, k2, p1, p2, k3):
[[-4.32166091e-01  2.83162078e-01 -2.23954456e-04  4.15174577e-04
  -1.38440979e-01]]

Reprojection Error: 0.0074 pixels


Calibration Results for assign1_data/Calibration/calib3/*.png

Intrinsic Matrix (K):
[[837.55334325   0.         338.44583063]
 [  0.         837.73977506 276.54574207]
 [  0.           0.           1.        ]]

Distortion Coefficients (k1, k2, p1, p2, k3)

### b)

In [ ]:
# Prepare object points for the chessboard
objp = np.zeros((PATTERN_SIZE[0] * PATTERN_SIZE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:PATTERN_SIZE[0], 0:PATTERN_SIZE[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE  # Scale to real-world coordinates (mm)

# Load the final setup image
img = cv.imread('assign1_data/Calibration/final_setup.png')

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Find chessboard corners
ret, corners = cv.findChessboardCorners(
    gray, 
    PATTERN_SIZE,
    cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE
)

# Refine corner positions
corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

# Solve for extrinsic parameters (rotation and translation vectors)
success, rvec, tvec = cv.solvePnP(objp, corners2, mtx, dist)

if success:
    # Convert rotation vector to rotation matrix using Rodrigues
    R, _ = cv.Rodrigues(rvec)
    
    # Construct the extrinsic matrix [R|t] (3x4)
    extrinsic_matrix = np.hstack((R, tvec))
    
    # Optional: Create 4x4 transformation matrix
    extrinsic_4x4 = np.vstack((extrinsic_matrix, [0, 0, 0, 1]))
    
    print("\n" + "="*60)
    print("EXTRINSIC CALIBRATION RESULTS")
    print("="*60)
    print("\nRotation Vector (Rodrigues):")
    print(rvec.flatten())
    print("\nRotation Matrix (R):")
    print(R)
    print("\nTranslation Vector (t) [mm]:")
    print(tvec.flatten())
    print("\nExtrinsic Matrix [R|t] (3x4):")
    print(extrinsic_matrix)
    print("\nExtrinsic Matrix (Homogeneous 4x4):")
    print(extrinsic_4x4)
    
    # Calculate pixel-to-millimeter conversion factor
    # Method 1: Using the known chessboard square size
    corner1 = corners2[0].flatten()
    corner2 = corners2[PATTERN_SIZE[0]].flatten()  # One square down
    
    pixel_distance = np.linalg.norm(corner1 - corner2)
    actual_distance_mm = SQUARE_SIZE  # mm
    
    mm_per_pixel = actual_distance_mm / pixel_distance
    
    # Method 2: Using Z-distance from camera
    Z = tvec[2][0]  # Distance to plane in mm
    focal_length_px = mtx[0, 0]  # fx from intrinsic matrix
    
    mm_per_pixel_z = Z / focal_length_px
    
    print("\n" + "="*60)
    print("PIXEL-TO-MILLIMETER CONVERSION")
    print("="*60)
    print(f"\nMethod 1 (Chessboard-based):")
    print(f"  Pixel distance between corners: {pixel_distance:.2f} px")
    print(f"  Actual distance: {actual_distance_mm} mm")
    print(f"  Conversion factor: {mm_per_pixel:.4f} mm/pixel")
    
    print(f"\nMethod 2 (Z-distance based):")
    print(f"  Distance to plane (Z): {Z:.2f} mm")
    print(f"  Focal length: {focal_length_px:.2f} px")
    print(f"  Conversion factor: {mm_per_pixel_z:.4f} mm/pixel")
    
    print("\n" + "="*60)
    
    # Optional: Visualize detected corners
    img_corners = img.copy()
    cv.drawChessboardCorners(img_corners, PATTERN_SIZE, corners2, ret)
    
    # Draw coordinate axes on the image
    axis_length = 3 * SQUARE_SIZE  # 3 squares length
    axis = np.float32([[axis_length, 0, 0],
                        [0, axis_length, 0],
                        [0, 0, -axis_length]]).reshape(-1, 3)
    
    imgpts, _ = cv.projectPoints(axis, rvec, tvec, mtx, dist)
    origin, _ = cv.projectPoints(np.float32([[0, 0, 0]]), rvec, tvec, mtx, dist)
    
    origin = tuple(origin[0].ravel().astype(int))
    img_corners = cv.line(img_corners, origin, tuple(imgpts[0].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    img_corners = cv.line(img_corners, origin, tuple(imgpts[1].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    img_corners = cv.line(img_corners, origin, tuple(imgpts[2].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)
    
    # Save the result
    cv.imwrite('extrinsic_calibration_result.png', img_corners)
    print("\n✓ Visualization saved as 'extrinsic_calibration_result.png'")
    
    cv.imshow('Extrinsic Calibration - Coordinate Axes', img_corners)
    cv.waitKey(0)
    cv.destroyAllWindows()


EXTRINSIC CALIBRATION RESULTS

Rotation Vector (Rodrigues):
[-0.0872788   0.10046696  3.1282892 ]

Rotation Matrix (R):
[[-0.9983912  -0.01225302 -0.0553613 ]
 [ 0.00867551 -0.99788611  0.06440541]
 [-0.05603343  0.0638215   0.99638701]]

Translation Vector (t) [mm]:
[ 64.16261762  35.67024177 266.73507476]

Extrinsic Matrix [R|t] (3x4):
[[-9.98391201e-01 -1.22530180e-02 -5.53613021e-02  6.41626176e+01]
 [ 8.67550629e-03 -9.97886105e-01  6.44054075e-02  3.56702418e+01]
 [-5.60334347e-02  6.38215048e-02  9.96387008e-01  2.66735075e+02]]

Extrinsic Matrix (Homogeneous 4x4):
[[-9.98391201e-01 -1.22530180e-02 -5.53613021e-02  6.41626176e+01]
 [ 8.67550629e-03 -9.97886105e-01  6.44054075e-02  3.56702418e+01]
 [-5.60334347e-02  6.38215048e-02  9.96387008e-01  2.66735075e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

PIXEL-TO-MILLIMETER CONVERSION

Method 1 (Chessboard-based):
  Pixel distance between corners: 45.49 px
  Actual distance: 15 mm
  Conversion factor: 

: 

## 2. Calibration images:

### a)

In [ ]:
# Load image
img = cv.imread('assign1_data/Isolated/colored_bricks.png')
img_copy = img.copy()

# Convert to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Apply Gaussian blur
blurred = cv.GaussianBlur(gray, (5, 5), 0)

# Threshold to segment blocks from background
_, thresh = cv.threshold(blurred, 200, 255, cv.THRESH_BINARY_INV)

# Find contours (each LEGO block)
contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

rois = []
print(f"Found {len(contours)} potential ROIs")

# Extract bounding boxes for each contour
for i, contour in enumerate(contours):
    # Filter small contours (noise)
    area = cv.contourArea(contour)
    if area < 500:  # Adjust threshold based on your image
        continue
    
    # Get bounding rectangle
    x, y, w, h = cv.boundingRect(contour)
    
    # Store ROI information
    rois.append({
        'id': i,
        'bbox': (x, y, w, h),
        'area': area
    })
    
    # Draw bounding box
    cv.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv.putText(img_copy, f"ROI {i}", (x, y-5), 
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Extract individual ROI
    roi = img[y:y+h, x:x+w]
    cv.imwrite(f'roi_block_{i}.jpg', roi)

print(f"Detected {len(rois)} LEGO blocks")
print("ROI information:", rois)

cv.imshow('All ROIs Detected', img_copy)
cv.imshow('Threshold', thresh)
cv.waitKey(0)
cv.destroyAllWindows()

Found 11 potential ROIs
Detected 2 LEGO blocks
ROI information: [{'id': 4, 'bbox': (105, 412, 58, 49), 'area': 1526.0}, {'id': 10, 'bbox': (0, 0, 640, 480), 'area': 286565.5}]


: 